In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
import sqlite3
from string import punctuation
from nltk import sent_tokenize, wordpunct_tokenize
import spacy
import re

nlp = spacy.load("ru_core_news_sm")

In [3]:
data = pd.read_csv('raw_data.csv')

In [4]:
data['text'].replace('', np.nan, inplace=True)

In [5]:
data.dropna(subset=['text'], inplace=True)

In [6]:
data.reset_index()

,index,text,topic,source
0,0,"Если кто-либо достоинНеземной любви,То я знаю,...",stihi,https://skynight.ru/stihi_love.html
1,1,"Спасибо за то, что ты есть.За то, что твой гол...",stihi,https://skynight.ru/stihi_love.html
2,2,"Среди таинственно-высоких гор,Вокруг кристальн...",stihi,https://skynight.ru/stihi_love.html
3,3,"Когда меня ты нежно обнимаешь,Я тут же забываю...",stihi,https://skynight.ru/stihi_love.html
4,4,"Люблю, скучаю, обожаю,Схожу с ума и умираюОт с...",stihi,https://skynight.ru/stihi_love.html
...,...,...,...,...
19015,19023,"– Послушай, я люблю тебя больше всех на свете....",book,"Стефани Майер, ""Сумерки"""
19016,19024,"– Да, – улыбнулся Эдвард. – Достаточно на сего...",book,"Стефани Майер, ""Сумерки"""
19017,19025,Холодные губы снова прильнули к моей шее.,book,"Стефани Майер, ""Сумерки"""
19018,19026,1,book,"Стефани Майер, ""Сумерки"""


In [7]:
def clean_text(text):
    '''функция для разлипания строк и нормальной построчной записи стихов, а еще для разлипания знаков препинания'''
    pattern1 = r'([.,!?–:;-]|[а-я])([А-Я])'
    repl1 = r'\1\n\2'
    #pattern2 = r'([.,!?–:;])(\S)'
    repl2 = r'\1 \2'
    pattern3 = r'(\.{3})([–.,!?–:;А-Я])'
    res = re.sub(pattern1, repl1, text)
    res = re.sub(pattern3, repl1, res)
    #res = re.sub(pattern2, repl2, res)
    return res

функция для препроцессинга (очистка от слипшихся символов и деление на предложения и строки)

In [8]:
def preprocessing(df):
    text = df['text']
    source = df['source']
    topic = df['topic']
    sentences_cl = text.split('\n')
    sentences = []
    for sent in sentences_cl:
        sent = clean_text(sent)
        pattern2 = r'([.,!?–:;…])(\S)'
        repl2 = r'\1\n\2'
        res = re.sub(pattern2, repl2, sent)
        sent = res.split('\n')
        for s in sent:
            sentences.extend(sent_tokenize(s))
    return (clean_text(text), sentences, topic, source)

функция для морфологической разметки и записи метаинформации о тексте

In [9]:
def tag(df, i):
    info = {}
    info['index'] = i
    prep = preprocessing(df)
    info['text'] = prep[0]
    info['source'] = prep[3]
    info['topic'] = prep[2]
    sentences = prep[1]
    text_sentences = []
    for sentence in sentences:
        if sentence != '..':
            sent_info = {}
            sent_info['sentence'] = sentence
            doc = nlp(sentence)
            words = []
            for token in doc:
                if token.pos_ != 'PUNCT' and token.text.isalpha(): #проверяем токен
                    word_info = {}
                    word_info['word'] = token.text.lower()
                    word_info['lemma'] = token.lemma_
                    if token.pos_ == 'AUX': #убираем теги AUX
                        word_info['pos'] = 'VERB'
                    else:
                        word_info['pos'] = token.pos_
                    words.append(word_info)
            sent_info['words'] = words
            text_sentences.append(sent_info)
    info['sentences'] = text_sentences
    return info

In [10]:
analysis = []

In [11]:
for i in range(0, len(data.index)):
    try:
        analysis.append(tag(data.iloc[i], i))
    except:
        print(data.iloc[i])
    

создаем базу данных: 

In [21]:
corpname = 'LOVECORP'

In [22]:
conn = sqlite3.connect(f'{corpname}.db')
cur = conn.cursor()
cur.execute("""
CREATE TABLE IF NOT EXISTS texts
(id INTEGER PRIMARY KEY AUTOINCREMENT, text text, topic text, source text)
""")
# таблица тексты: ключ, текст, источник
# таблица предложения: номер текста, текст предложения
cur.execute("""
CREATE TABLE IF NOT EXISTS sentences 
(id INTEGER PRIMARY KEY AUTOINCREMENT, text_id int, sentence text) 
    """)
# таблика токен: позиция в предложении, номер текста, номер предложения, токен(=слово как встретилось), лемма, часть речи
cur.execute("""
CREATE TABLE IF NOT EXISTS token
(id INTEGER PRIMARY KEY AUTOINCREMENT, text_id int, sentence_id int, token text, lem text, pos text) 
""")

conn.commit()
conn.close()

функция для записи в БД:

In [23]:
conn = sqlite3.connect(f'{corpname}.db', timeout=10)
cur = conn.cursor()
def write_to_db(info):
    text = info['text']
    text_id = info['index'] + 1
    topic = info['topic']
    source = info['source']
    cur.execute('INSERT INTO texts VALUES (?, ?, ?, ?)', (text_id, text, topic, source))
    conn.commit()
    sent_id = text_id * 1000 #танцы с бубном, чтобы id не повторялись, но было видно, какое это по счету предложение текста
    for item in info['sentences']:
        sentence = item['sentence']
        cur.execute('INSERT INTO sentences VALUES (?, ?, ?)', (sent_id, text_id, sentence))
        conn.commit()
        word_num = sent_id * 10000 
        for word in item['words']:
            token = word['word']
            lemma = word['lemma']
            pos = word['pos']
            cur.execute('INSERT INTO token VALUES (?, ?, ?, ?, ?, ?)', (word_num, text_id, sent_id, token, lemma, pos))
            conn.commit()
            word_num += 1
        sent_id += 1

пишем данные в БД:

In [24]:
for el in analysis:
    write_to_db(el)
conn.close()